In [ ]:
import pandas as pd
from sqlalchemy import create_engine
import dateparser
import numpy as np

pd.set_option('display.width', 400)
pd.set_option('display.max_columns', 11)

# Credentials to connect to the database
username = "username"
password = "DB_password"
hostname = "DB_host"
dbname = "DB_name"

# Process the initial data
def first_data_handle():
    engine = create_engine(f"mysql+mysqlconnector://{username}:{password}@{hostname}/{dbname}")
    data = pd.read_sql_table("sales2", engine)
    data.product_price = round(data.product_price, 2)
    zero_price_data = data[data.product_price == 0.0]
    data.drop(zero_price_data.index, axis=0, inplace=True)
    data.index = range(len(data))
    with pd.option_context('mode.use_inf_as_na', True):
        data = data.dropna(axis=0)
    data.index = range(len(data))
    data.to_sql(name="sales", con=engine, if_exists="replace", index=False, chunksize=1000)

# Process the data and find the products that were purchased more than num_of_sales
def products_with_sales(num_of_sales=1000):
    engine = create_engine(f"mysql+mysqlconnector://{username}:{password}@{hostname}/{dbname}")
    data = pd.read_sql_table("sales", engine)
    data["order_timestamp"] = data["order_timestamp"].astype("str")
    products_with_high_vol = data.groupby("product_id").product_quantity.sum()
    products_with_high_vol = products_with_high_vol[products_with_high_vol >= num_of_sales]
    data = data.set_index("product_id").join(products_with_high_vol, rsuffix="_total")
    data.dropna(axis=0, inplace=True)
    data.drop(columns="product_quantity_total", inplace=True)
    data.sort_values(by="order_timestamp", inplace=True)
    first_date = dateparser.parse(data.iloc[0].order_timestamp)
    last_date = dateparser.parse(data.iloc[-1].order_timestamp)
    shift = 6 - ((last_date - first_date).days % 7)
    data["week"] = (((data["order_timestamp"].apply(dateparser.parse) - first_date).dt.days + shift) // 7) + 1
    data.reset_index(inplace=True)
    data.to_sql(name=f"products_{num_of_sales}_sales", con=engine, index=False, if_exists="replace", chunksize=10)

# Data aggregation. Collect the data on a weekly basis
def create_week_data():
    engine = create_engine(f"mysql+mysqlconnector://{username}:{password}@{hostname}/{dbname}")
    data = pd.read_sql_table("products_1000_sales", engine)
    data["order_timestamp"] = data["order_timestamp"].astype("str")
    demand_data = data.groupby(["product_id", "week"]).product_quantity.sum()
    price_data = data.groupby(["product_id", "week"]).product_price.mean().round(2)
    week_data = pd.concat([demand_data, price_data], axis=1).reset_index()
    products = week_data.product_id.unique()
    cost = pd.Series(dtype=float)
    max_prices = pd.Series(dtype=float)
    for product in products:
        min_price = week_data.loc[week_data.product_id == product].product_price.min()
        max_price = week_data.loc[week_data.product_id == product].product_price.max()
        temp_ind = week_data.loc[week_data.product_id == product].index
        for i in temp_ind:
            cost.at[i] = round(0.8 * min_price, 2)
            max_prices.at[i] = round(1.2 * max_price, 2)
    week_data["product_cost"] = cost
    week_data["product_max_bound"] = max_prices
    week_data.to_sql(name="week_data", con=engine, index=False, if_exists="replace", chunksize=1000)

def full_weeks(missing_weeks, total_weeks):
    full_weeks = pd.DataFrame(columns=missing_weeks.columns)
    for week in range(1, total_weeks+1):
        flag = week in missing_weeks["week"].values
        temp = [
            missing_weeks.loc[0, "product_id"],
            week,
            missing_weeks.loc[missing_weeks["week"] == week, "product_quantity"].values[0] if flag else 0,
            missing_weeks.loc[missing_weeks["week"] == week, "product_price"].values[0] if flag else 0,
            missing_weeks.loc[0, "product_cost"],
            missing_weeks.loc[0, "product_max_bound"]
        ]
        full_weeks.loc[week - 1] = temp
    return full_weeks

def nn_row(row, full_weeks, number_of_weeks):
    row_data = [
        row["week"], row["product_cost"], row["product_max_bound"], row["product_id"]
    ]
    weeks = np.zeros((number_of_weeks, 2))
    for i in range(1, number_of_weeks+1):
        temp_week = row["week"] - i
        if temp_week >= 1:
            p = full_weeks.loc[full_weeks["week"] == temp_week, "product_price"].values[0]
            q = full_weeks.loc[full_weeks["week"] == temp_week, "product_quantity"].values[0]
        else:
            p = q = 0
        weeks[number_of_weeks - i, :] = [p, q]
    row_data.extend(weeks.flatten())
    row_data.extend([row["product_price"], row["product_quantity"]])
    return row_data

def create_nn_data(number_of_weeks=16):
    engine = create_engine(f"mysql+mysqlconnector://{username}:{password}@{hostname}/{dbname}")
    week_data = pd.read_sql_table("week_data", engine)
    total_weeks = week_data.week.max()
    columns = ["week", "product_cost", "product_max_bound", "product_id"]
    columns.extend(f"P{i}" for i in range(1, number_of_weeks+2))
    columns.extend(f"Q{i}" for i in range(1, number_of_weeks+2))
    nn_data = pd.DataFrame(columns=columns)
    for product in week_data.product_id.unique():
        temp_product = week_data[week_data["product_id"] == product].copy()
        temp_product.reset_index(drop=True, inplace=True)
        full_week = full_weeks(temp_product, total_weeks)
        temp_data = pd.DataFrame(columns=columns)
        for _, row in temp_product.iterrows():
            temp_data.loc[len(temp_data)] = nn_row(row, full_week, number_of_weeks)
        nn_data = pd.concat([nn_data, temp_data], ignore_index=True)
    nn_data.to_sql(name="nn_data", con=engine, index=False, if_exists="replace", chunksize=1000)

def pso_data():
    engine = create_engine(f"mysql+mysqlconnector://{username}:{password}@{hostname}/{dbname}")
    nn_data = pd.read_sql_table("nn_data", engine)
    total_weeks = nn_data.week.max()
    number_of_weeks = (nn_data.shape[1] - 6) // 2
    data = nn_data[nn_data["week"] == total_weeks].copy()
    data.index = range(len(data))
    for i in range(1, number_of_weeks+1):
        data[f"P{i}"] = data[f"P{i+1}"]
        data[f"Q{i}"] = data[f"Q{i+1}"]
    data.drop(columns=["week", f"P{number_of_weeks+1}", f"Q{number_of_weeks+1}"], inplace=True)
    pso = data.copy()
    pso["product_min_bound"] = 0
    opt_data = pd.read_sql_table("data_for_optimization", engine)
    for product in opt_data["product_id"]:
        temp = data[data["product_id"] == product].copy()
        arxiki = opt_data[opt_data["product_id"] == product].arxikiTimi.values[0]
        teliki = opt_data[opt_data["product_id"] == product].telikiTimi.values[0]
        percentage = 1 - (teliki / arxiki)
        max_bound = min((1 - (percentage - 0.1)) * arxiki, arxiki)
        min_bound = (1 - (percentage + 0.1)) * arxiki
        temp["product_max_bound"] = round(float(max_bound), 2)
        temp["product_min_bound"] = round(float(min_bound), 2)
        pso = pd.concat([pso, temp], ignore_index=True)
    pso.to_sql(name="pso_data", con=engine, index=False, if_exists="replace", chunksize=10)

# Helpful function to see the details of a dataframe
def print_details(data):
    print(f"Number of customers: {data.customer_id.nunique()}")
    print(f"Number of orders: {data.order_id.nunique()}")
    print(f"Number of products: {data.product_id.nunique()}")
    print(f"First date: {data.order_timestamp.min()}")
    print(f"Last date: {data.order_timestamp.max()}")
    print("Columns in data:")
    print(list(data.columns))
    print(data.shape)
    print(data.head(10))
    print(data.tail(10))
